***
### <span style='color:green'> ME Algorithm  &emsp;&emsp; Feb, 2024 </span>
### <span style='color:Blue'> Phase 3 </span>
### <p> Yan-Bin Chen (陳彥賓) &emsp; yanbin@ntu.edu.tw </p>
### <p> Master Program in Statistics, National Taiwan University, Taipei, Taiwan.</p> 
---

# Input

In [1]:
import pickle
import numpy as np
import collections
import pandas
import itertools
from itertools import chain
import random 
import time
import datetime


In [2]:
# Input
PATH1='../data/seedinds.txt'
PATH2='../data/bilabels.txt'
PATH3='../data/seedinds_neighborregions.txt'
PATH4='../data/ResNet18_PlantDisease_45K_Spec200.csv'
PATH5='../data/ResNet18_PlantDisease_45K_Values.csv'


#===  parameters ========================================================
TRIALS    = 1
timestr   = ''
REG_COLUMN = "Spec200"
RAW_2D_DATA = False

if RAW_2D_DATA: # 2D
    from CNN_Modules import ME_CNN
else: # 1D
    from CNN_Modules_1D import ME_CNN


# (1)PATH1
df1 = pandas.read_csv(PATH1, header=None, delimiter = "\t")
region = df1.to_numpy().T[0]
NUM_region = len(region)

# (2)PATH2
df2 = pandas.read_csv(PATH2, header=None, delimiter = "\t")
cen = df2.to_numpy()

print('region:', region)
print('region amount:', len(region))


# (4)PATH4. Have to be here. The following neighboring process needs this information
df4 = pandas.read_csv(PATH4)
display(df4.head())
#all_region_index  = df.iloc[:,REGION_INDEX_LOC].to_numpy().astype(int)
all_region_index  = df4[REG_COLUMN].to_numpy().astype(int)
print("all_region_index\n",len(all_region_index), all_region_index)


# (4)PATH4 reset label
label=df4['Label'].value_counts(sort=False).keys()
_=[df4['Label'].replace(to_replace=label[i], value=i, inplace=True) for i in range(len(label))]
display(df4['Label'].value_counts(sort=False))
test_label_answer = df4["Label"].to_numpy()

region: [ 33  95 131  24 125  10  94 192 135  25 139 168 110  60 134  27  58  28]
region amount: 18


,X1,X2,X3,Class,Label,Spec200
0,-6.131136,-11.714952,14.559869,Cherry,2,100
1,4.653914,-15.913769,8.271565,Cherry,2,6
2,4.666949,-15.911179,8.316633,Cherry,2,6
3,-1.160013,-8.344077,19.694381,Cherry,2,166
4,-0.430728,-5.483346,17.380125,Cherry,2,170


all_region_index
 43217 [100   6   6 ... 186  83  32]


0    5000
1    4386
2    5000
3    5000
4    4457
5    4876
6    5000
7    4498
8    5000
Name: Label, dtype: int64

In [3]:
df1
import pandas as pd
print(pd.__version__)

1.1.3


In [4]:
# (5)PATH5     convert the embedded data into the pickle file
df5 = pandas.read_csv(PATH5)
test_array = df5.to_numpy()


#save
PATH5='../data/embedded_data.pickle'   #replace original PATH 5
with open(PATH5, 'wb') as f:
    pickle.dump([test_array, test_label_answer], f)
print(test_label_answer, "\n", test_array)

[1 1 1 ... 8 8 8] 
 [[ -6.131136   -11.71495238  14.55986935]
 [  4.65391405 -15.91376892   8.27156454]
 [  4.66694856 -15.91117949   8.31663274]
 ...
 [ -1.49778226  27.08358869 -12.30754204]
 [ -0.53695937  14.94667994  -7.44467661]
 [ -6.71398626   0.54063622   0.42243439]]


# Get neighbors

In [5]:
if (PATH3):
    df = pandas.read_csv(PATH3, delim_whitespace=' ', header=None,  index_col=0)
    neighbors = df.to_numpy()
    NUM_NEI=df.shape[1]
    
    print('neighbor amount: ',len(list(chain.from_iterable(neighbors))))
    print(neighbors)

# filter out duplicated ones
    test_list=list(chain.from_iterable(neighbors))
    res2=[]
    [res2.append(n) for n, i in enumerate(test_list) if i in test_list[:n]]

    res2.reverse()
    neighbors=neighbors.tolist()
    [neighbors[x//NUM_NEI].pop(x%NUM_NEI) for x in res2]
    print('neighbor amount: ',len(list(chain.from_iterable(neighbors))))
    display(neighbors)

neighbor amount:  90
[[138  55  48  87   1]
 [ 71 158 144  81  69]
 [ 80  65  77  57 164]
 [ 15  63 146 115 109]
 [ 68 127  36  67 174]
 [197 120  96 130  50]
 [169 145 137 151  72]
 [116 147  91  64 198]
 [165  37  73 117 171]
 [ 17 156 175  88  66]
 [185 162  21 136  13]
 [113  20  84 199  16]
 [ 54 108 111  52   7]
 [ 61 160 121  26 146]
 [155 161  41  86 157]
 [178  56   5  42  75]
 [126 180  89 179  72]
 [ 59 153  44 167  18]]
neighbor amount:  88


[[138, 55, 48, 87, 1],
 [71, 158, 144, 81, 69],
 [80, 65, 77, 57, 164],
 [15, 63, 146, 115, 109],
 [68, 127, 36, 67, 174],
 [197, 120, 96, 130, 50],
 [169, 145, 137, 151, 72],
 [116, 147, 91, 64, 198],
 [165, 37, 73, 117, 171],
 [17, 156, 175, 88, 66],
 [185, 162, 21, 136, 13],
 [113, 20, 84, 199, 16],
 [54, 108, 111, 52, 7],
 [61, 160, 121, 26],
 [155, 161, 41, 86, 157],
 [178, 56, 5, 42, 75],
 [126, 180, 89, 179],
 [59, 153, 44, 167, 18]]

In [6]:
# combine seed regions and neighbors
if (PATH3):
    reg_nei=[]
    for i in range(len(neighbors)):
        a=[region[i]]
        b=neighbors[i]
        if len(b):
            c=list(np.concatenate((a,b),axis=0))
        else:
            c=a.copy()
        reg_nei.append(c)
else:
    reg_nei=region.copy()

reg_nei

[[33, 138, 55, 48, 87, 1],
 [95, 71, 158, 144, 81, 69],
 [131, 80, 65, 77, 57, 164],
 [24, 15, 63, 146, 115, 109],
 [125, 68, 127, 36, 67, 174],
 [10, 197, 120, 96, 130, 50],
 [94, 169, 145, 137, 151, 72],
 [192, 116, 147, 91, 64, 198],
 [135, 165, 37, 73, 117, 171],
 [25, 17, 156, 175, 88, 66],
 [139, 185, 162, 21, 136, 13],
 [168, 113, 20, 84, 199, 16],
 [110, 54, 108, 111, 52, 7],
 [60, 61, 160, 121, 26],
 [134, 155, 161, 41, 86, 157],
 [27, 178, 56, 5, 42, 75],
 [58, 126, 180, 89, 179],
 [28, 59, 153, 44, 167, 18]]

In [7]:
# collect images from target region and neighboring regions
# input : region, NUM_region, cen, all_region_index, neighbors
# output: region_image: to save image indices corresponding to seed regions.
#         region_answer: to save answer

region_image_before=[]
region_image=[]
region_image_pure=[]
for i in range(NUM_region):
    
    
    #(1)neighbor  nei
    if (PATH3):
        addr_nei=[]
        for j in range(len(neighbors[i])):
            addr_nei=addr_nei+list(np.where(all_region_index==neighbors[i][j])[0])
            #check whether it has duplicates
            if (len(addr_nei) != len(set(addr_nei))):
                print("neighbor duplicate at i=",i,"j=",j)
                addr_nei=list(set(addr_nei))

    #(2)original
    addr=list(np.where(  (all_region_index==region[i])  &  (cen.T[1]==1)  )[0])
    if (PATH3):
        addr=addr+addr_nei
    #check whether it has duplicates
    if (len(addr) != len(set(addr))):
        print("** original duplicate at i=",i,";region",region[i],";duplicate size",len(addr)-len(set(addr)))
        addr=list(set(addr))
    region_image.append(addr)

    #(3)original before centroid (this is only for check, rather than for main codes)
    addr_before=list(np.where(all_region_index==region[i])[0])
    if (PATH3):
        addr_before=addr_before+addr_nei
    #check whether it has duplicates
    if (len(addr_before) != len(set(addr_before))):
        print("** same duplicate situation")
        addr_before=list(set(addr_before))
    region_image_before.append(addr_before)

    #(4)pure
    addr_pure=list(np.where(  (all_region_index==region[i])  &  (cen.T[1]==1)  )[0])
    region_image_pure.append(addr_pure)


with open('../data/' + timestr + 'region_for_phase5.pickle', 'wb') as f:
    pickle.dump([region, reg_nei, region_image_pure, region_image,], f)

In [8]:
# ==== test_array ====
with open(PATH5, 'rb') as f:
    test_array, test_label_answer = pickle.load(f)
print("test_array:",np.shape(test_array))


# 1213 add auto judge /255
# 20240319
if RAW_2D_DATA: # 2D
    print("")
else: # 1D
    test_array = np.expand_dims(test_array, axis = -1)
test_array /= 255
print("training size:",len(np.array(list(chain.from_iterable(region_image)))))

test_array: (43217, 3)
training size: 22653


# CNN original

In [ ]:
# ==== CNN Original =====
ROUND_start = time.time()

for n in range(5):
    region=region_image.copy()
    region=list(region)
    selected_region = list(range(NUM_region))

    p1=0
    p2=0

    print("n, p1, p2", n, p1, p2)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))


    # fill up training array
        
    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)
        


    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_n0_R' + str(p1) + '+R'+ str(p2) +'_trial' + str(n)+'.pickle'  #extra_original
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)

In [ ]:
#==== shift label =======
N=5
Original_result=[]
Original_prob=[]
for i in range(N):
    with open('./' + timestr + '(classes=' + str(NUM_region) + ')_n0_R0+R0_trial' + str(i)+ '.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    #(1)
    Original_result.append(label_B)

    #(2)
    Original_prob.append(label_B_prob)
print(np.shape(Original_result), np.shape(Original_prob))

# save pickle
with open('./' + timestr + 'results_of_original.pickle', 'wb') as f:
    pickle.dump([Original_result, Original_prob], f)
    
# save mat
from scipy.io import savemat
savemat('./' + timestr + 'results_of_original.mat', {'result_for_original':Original_result, 'prob_for_original':Original_prob})

# CNN combination and removal

In [ ]:
#====CNN combination c(20,2)======
comb=[]
for subset in itertools.combinations(range(NUM_region), 2):
    comb.append(subset)
NUM_comb=len(comb)
display(NUM_comb)            

for n in range(NUM_comb+1):
    region=region_image.copy()
    region=list(region)
    selected_region = list(range(NUM_region))

    if (n > 0):
        p1=comb[n-1][0]
        p2=comb[n-1][1]
        region[p1]=region[p1]+region[p2]
        region.pop(p2)
        selected_region.pop(-1)
    else:
        p1=0
        p2=0

    print("n, p1, p2", n, p1, p2)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)


    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_n' + str(n) + '_R' + str(p1) + '+R'+ str(p2) +'.pickle'
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)



# ==== CNN Removal =====
for n in range(NUM_region):
    region=region_image.copy()   #reset
    selected_region = list(range(NUM_region))
    selected_region.pop(n)

    print("n=", n)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))


    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)

    train_array = np.expand_dims(train_array, axis = -1)
    
    
    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_Remove' + str(n) +'.pickle' 
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)
  

In [ ]:
# ==== CNN Removal =====
for n in range(NUM_region):
    region=region_image.copy()   #reset
    selected_region = list(range(NUM_region))
    selected_region.pop(n)

    print("n=", n)


    # ===== one CNN =============
    NUM_CLASSES = len(selected_region)

    # input image and label
    Input_img     = []
    Input_img_len = []
    for c,sel in enumerate(selected_region, start=0):
        Input_img = Input_img + list(region[sel])
        Input_img_len.append(len(region[sel]))

    # 20240319
    if RAW_2D_DATA: # 2D
        W           = np.shape(test_array[0])[0]
        H           = np.shape(test_array[0])[1]
        train_array = np.zeros((len(Input_img), W, H), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W,H)
    else: # 1D
        W           = np.shape(test_array[0])[0]
        train_array = np.zeros((len(Input_img), W), dtype=float)
        for i in range (len(Input_img)):
            train_array[i] = test_array[Input_img[i]].reshape(W)
            
    train_array = np.expand_dims(train_array, axis = -1)
    
    
    # fill up the training label to each training image
    current_train_label = np.zeros(len(train_array), dtype=int)
    accum_base=0
    for label in range(1,NUM_CLASSES):
        pre_cursor = Input_img_len[label-1]
        accum_base = accum_base + pre_cursor
        current_train_label[accum_base:] = label

    # CNN
    #===============================================
    one_predicted_results  = np.zeros((TRIALS, len(test_label_answer)), dtype=int)
    one_predict_percentage = np.zeros((TRIALS, len(test_label_answer), NUM_CLASSES), dtype=float)    
    model_history = np.zeros(TRIALS, dtype=list)

    for r in range(TRIALS):
        one_predicted_results[r], one_predict_percentage[r], model_history[r] = ME_CNN(
                x_train     = train_array,
                train_label = current_train_label,
                test_array  = test_array,
                true_answer = test_label_answer,
                Num_Classes = NUM_CLASSES
                )
        print(type(model_history))


        # ===== delete CNN tensors =====
        from keras import backend as K
        K.clear_session()
        import gc
        gc.collect()

        print("One CNN, r: ",r)
        ROUND_duration = time.time() - ROUND_start
        print("Computing Time: ", str(datetime.timedelta(seconds=ROUND_duration)))

    # === save to file ===
    savefile_path = './' + str(timestr) + '(classes=' + str(NUM_CLASSES)+')_Remove' + str(n) +'.pickle' 
    with open(savefile_path, 'wb') as f:
        pickle.dump([Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history], f)


In [ ]:
# ====shift label=====
#=== combination =====
comb=[]
for oneset in itertools.combinations(range(NUM_region), 2):
    comb.append(oneset)
NUM_comb=len(comb)

Merged_result=[]
Merged_prob=[]
Merged_prob_label=[]

for n in range(NUM_comb):
    label = list(range(NUM_region))
    p1=comb[n][0]
    p2=comb[n][1]

    with open('./' + timestr + '(classes=' + str(NUM_region-1) + ')_n'+str(n+1)+'_R'+str(p1)+'+R'+str(p2)+'.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    #(1)shift label index
    for p in reversed(range(p2,NUM_region-1)):
        addr=np.where(label_B==p)[0]
        label_B[addr]+=1
    Merged_result.append(label_B)

    #(2)
    Merged_prob.append(label_B_prob)

    #(3)
    label.pop(p2)
    Merged_prob_label.append(label)

print(np.shape(Merged_result), np.shape(Merged_prob), np.shape(Merged_prob_label))

# save pickle
with open('./' + timestr + 'results_of_combination.pickle', 'wb') as f:
    pickle.dump([comb, Merged_result, Merged_prob, Merged_prob_label], f)

# save mat
from scipy.io import savemat
if (np.shape(Merged_prob)[0]<=300):
    savemat('./' + timestr + 'results_of_combination.mat', {'combination_pairs':comb, 'result_for_merge':Merged_result, 'prob_for_merge':Merged_prob,'prob_label_for_merge': Merged_prob_label})
    print("normal size = ", np.shape(Merged_prob))
else:
    savemat('./' + timestr + 'results_of_combination.mat', {'combination_pairs':comb, 'result_for_merge':Merged_result, 'prob_label_for_merge': Merged_prob_label})
    savemat('./' + timestr + 'results_of_combination_prob1.mat', {'prob_for_merge1':Merged_prob[:200]})
    savemat('./' + timestr + 'results_of_combination_prob2.mat', {'prob_for_merge2':Merged_prob[200:400]})
    savemat('./' + timestr + 'results_of_combination_prob3.mat', {'prob_for_merge3':Merged_prob[400:]})
    print("large size = ", np.shape(Merged_prob))

    
    
    
#======== removal ===========
Removal_result=[]
Removal_prob=[]
Removal_prob_label=[]

for n in range(NUM_region):
    label = list(range(NUM_region))    
    #reset
    with open('./' + timestr + '(classes=' + str(NUM_region-1) + ')_Remove' + str(n) + '.pickle', 'rb') as f:
        Input_img, Input_img_len, one_predicted_results, one_predict_percentage, model_history = pickle.load(f)
    label_B = one_predicted_results[0].copy()
    label_B_prob = one_predict_percentage[0].copy()

    for p in reversed(range(n,NUM_region-1)):
        addr=np.where(label_B==p)[0]
        label_B[addr]+=1
    Removal_result.append(label_B)

    #(2)
    Removal_prob.append(label_B_prob)

    #(3)
    label.pop(n)
    Removal_prob_label.append(label)

print(np.shape(Removal_result), np.shape(Removal_prob), np.shape(Removal_prob_label))

# save pickle
with open('./' + timestr + 'results_of_removal.pickle', 'wb') as f:
    pickle.dump([Removal_result, Removal_prob, Removal_prob_label], f)
    
# save mat
from scipy.io import savemat
savemat('./' + timestr + 'results_of_removal.mat', {'result_for_removal':Removal_result,'prob_for_removal':Removal_prob, 'prob_label_for_removal':Removal_prob_label})

In [14]:
print("done")

done
